Defining Quic server and its functions.


In [1]:
import socket
import struct
import time
import math
import select

class QuicServer:
    local_port = 12345
    packet_size = 1024

    def __init__(self):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind(('', self.local_port))
        self.stream_data = {}
        self.start_time = {}
        self.total_bytes_received = 0
        self.total_packets_received = 0
        self.streams_complete = set()

    def process_initial_message(self):
        initial_message, sender_address = self.socket.recvfrom(4096)  # Increase buffer size to 4096 bytes
        if len(initial_message) < 8:
            raise ValueError("Initial message is too short")
        sender_ip, _ = sender_address
        num_streams, file_size = struct.unpack("!II", initial_message)
        return sender_ip, num_streams, file_size


    def update_statistics(self, stream_id, data):
        if stream_id not in self.stream_data:
            self.stream_data[stream_id] = {"bytes_received": 0, "packets_received": 0}
            self.start_time[stream_id] = time.time()
        self.stream_data[stream_id]["bytes_received"] += len(data)
        self.stream_data[stream_id]["packets_received"] += 1

        self.total_bytes_received += len(data)
        self.total_packets_received += 1

    def receive_packets(self, stream_id, file_size):
        received_data = b""
        while True:
            ready, _, _ = select.select([self.socket], [], [], 1)  # Wait for 1 second for incoming data
            if not ready:
                break  # Timeout reached, exit loop
            data, _ = self.socket.recvfrom(self.packet_size + 4)  # 4 bytes for stream_id
            received_stream_id = struct.unpack("!I", data[:4])[0]
            if received_stream_id != stream_id:
                continue  # Ignore data for other streams
            if data[4:] == b"END":
                break  # End of stream
            received_data += data[4:]
            self.update_statistics(stream_id, data[4:])
        return received_data

    def calculate_metrics(self):
        metrics = {}
        current_time = time.time()
        for stream_id, data in self.stream_data.items():
            elapsed_time_ms = (current_time - self.start_time.get(stream_id, current_time)) * 1000
            if elapsed_time_ms == 0:
                elapsed_time_ms = 1
            metrics[stream_id] = {
                "bytes_transferred": data["bytes_received"],
                "packets_transferred": data["packets_received"],
                "average_bytes_per_millisecond": data["bytes_received"] / elapsed_time_ms,
                "average_packets_per_millisecond": data["packets_received"] / elapsed_time_ms,
            }
        total_elapsed_time_ms = (current_time - self.start_time[1]) * 1000 if self.start_time else 1
        if total_elapsed_time_ms == 0:
            total_elapsed_time_ms = 1
        total_metrics = {
            "total_bytes_per_millisecond": self.total_bytes_received / total_elapsed_time_ms,
            "total_packets_per_millisecond": self.total_packets_received / total_elapsed_time_ms
        }
        return metrics, total_metrics

    def print_statistics(self, metrics, total_metrics, num_completed_streams, total_streams, file_size):
        print(f"Streams: {num_completed_streams} / {total_streams}")
        print(f"File size: {file_size}")

        for stream_id, metric in metrics.items():
            print(f"\nStream {stream_id}:")
            print(f"Bytes transferred: {metric['bytes_transferred']}")
            print(f"Packets transferred: {metric['packets_transferred']}")
            print(f"Average bytes per millisecond: {metric['average_bytes_per_millisecond']}")
            print(f"Average packets per millisecond: {metric['average_packets_per_millisecond']}")

        print("\nTotal Metrics:")
        print(f"Total bytes per millisecond: {total_metrics['total_bytes_per_millisecond']}")
        print(f"Total packets per millisecond: {total_metrics['total_packets_per_millisecond']}")


# Usage example:
server = QuicServer()
sender_ip, num_streams, file_size = server.process_initial_message()
for stream_id in range(1, num_streams + 1):
    server.receive_packets(stream_id, file_size)
metrics, total_metrics = server.calculate_metrics()
server.print_statistics(metrics, total_metrics, num_streams, 10, file_size)

Streams: 8 / 10
File size: 1540820

Stream 1:
Bytes transferred: 755712
Packets transferred: 738
Average bytes per millisecond: 90.21807541014654
Average packets per millisecond: 0.08810358926772123

Total Metrics:
Total bytes per millisecond: 90.21807541014654
Total packets per millisecond: 0.08810358926772123


Creating the server and receiving data.

The server receives an initial message that includes the amount of streams and the size of the file being transferred.
The server uses that information to appropriately receive the file over multiple streams.
For each stream, the server receives the file while updating the statistics.

In [2]:
# Create QuicServer instance
server = QuicServer()

# Receive the initial message and extract the sender's IP, number of streams, and file size
sender_ip, num_streams, file_size = server.process_initial_message()

# Process data for each stream and assemble the file
for stream_id in range(1, num_streams + 1):
    server.update_statistics(stream_id, b"")  # Initialize data for the stream
    received_data = server.receive_packets(stream_id, file_size)

# Calculate metrics for each stream and total metrics
metrics, total_metrics = server.calculate_metrics()



error: unpack requires a buffer of 8 bytes

Printing the statistics

Using the statistics calculated in the previous step, print the statistics for each stream and the total metrics.

In [ ]:
# Print statistics for each stream and total metrics
server.print_statistics(metrics, total_metrics)